# Master-Thesis 

Summary:  
Find mysterious soil property corresponding to soil-moisture behaviours. First Idea is to look at soil-moisture content after a minimum of 10 mm precipitation occured. Analyse the drying behaviours of the soil in the dry period after the rain event. 

# Table-of-content  

[AOI](#AOI)  



## Modul imports

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=2) # for printing pretty idk what it is... print with pp.pprint(print stuff)
import ipyleaflet
from ipyleaflet import Map, GeoJSON, basemaps, Popup, LayersControl
from ipywidgets import HTML
from IPython.display import Image, IFrame
import ee
ee.Initialize()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import dates
import altair as alt
from sklearn import cluster
from skimage import io, morphology, measure
from osgeo import gdal, gdal_array
gdal.UseExceptions()
gdal.AllRegister()


In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

## Functions

In [ ]:
#Get TileLayerurl from ee to plot on ipyleaflet
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

def clip_aoi(ImageCollection):
    im = ImageCollection.clip(ried_225_222)
    return im

# AOI

The area of interest is the hessisches Ried. The shape is digitized from a screnshot taken from http://atlas.umwelt.hessen.de/servlet/Frame/atlas/naturschutz/naturraum/karten/m_3_2_1.htm . I found no better source. Info: The HLNUG is selling this shapes there's no open source map.

In [ ]:
# Load AOI from GEE as FeatureCollection
ried_225_222 = ee.FeatureCollection('users/tillmueller1990/ried_225_222') # Use this but the boundary isn't that precise at borders, got it from wms layer 

#Create Hessen AOI
hesse_JSON = {"type":"Polygon","coordinates":[[[9.4988,51.6315],[9.628,51.6373],[9.6407,51.6081],[9.689,51.5831],[9.6484,51.5495],[9.6128,51.5563],[9.6253,51.5344],[9.5904,51.5228],[9.6447,51.4723],[9.6277,51.4536],[9.6462,51.4208],[9.5717,51.3932],[9.5607,51.378],[9.5783,51.3711],[9.5555,51.363],[9.5618,51.3484],[9.7312,51.3008],[9.7655,51.3129],[9.7707,51.3425],[9.7361,51.3233],[9.7059,51.3705],[9.8013,51.3899],[9.7949,51.4082],[9.8194,51.4156],[9.8442,51.4047],[9.8508,51.3754],[9.8693,51.3754],[9.8562,51.4158],[9.9112,51.4198],[9.9366,51.394],[9.9249,51.3718],[9.9311,51.3421],[9.9492,51.3346],[9.9433,51.3086],[9.9734,51.301],[9.9735,51.2862],[10.0573,51.2783],[10.0516,51.256],[10.0756,51.2485],[10.0759,51.2262],[10.1415,51.2221],[10.1536,51.2072],[10.237,51.1881],[10.2078,51.1512],[10.2198,51.1437],[10.2142,51.1178],[10.1964,51.1142],[10.1722,51.1514],[10.1247,51.1406],[10.1785,51.118],[10.1492,51.0885],[10.1614,51.0699],[10.1497,51.0551],[10.1913,51.0512],[10.2213,51.0251],[10.1977,51.0215],[10.2096,51.014],[10.198,50.9992],[10.1565,50.9958],[10.1325,51.0107],[10.0429,51.0149],[10.0195,50.9817],[10.0436,50.9704],[10.0318,50.9631],[10.044,50.9482],[10.0679,50.948],[10.0143,50.9372],[9.9484,50.9484],[9.9487,50.9262],[9.9904,50.9372],[9.9668,50.915],[9.9789,50.9076],[10.0145,50.9223],[10.0629,50.885],[10.0214,50.8667],[10.0216,50.8519],[10.0395,50.8555],[10.022,50.8333],[9.9739,50.8372],[9.9502,50.8224],[9.9567,50.7817],[9.9267,50.7817],[9.9209,50.7632],[9.9389,50.7596],[9.939,50.7374],[9.9032,50.7079],[9.9151,50.6969],[9.8735,50.6747],[9.8731,50.6418],[9.9461,50.6301],[9.964,50.6448],[9.9455,50.6599],[9.9515,50.6709],[10.049,50.6773],[10.0861,50.6324],[10.0863,50.6212],[10.0619,50.6289],[10.0382,50.614],[10.0507,50.6025],[10.0451,50.5723],[10.063,50.5573],[10.0396,50.5347],[10.04,50.4933],[9.9646,50.4254],[9.8473,50.3983],[9.7993,50.4243],[9.7578,50.4241],[9.7304,50.3558],[9.7427,50.3447],[9.7386,50.2998],[9.6528,50.2693],[9.6367,50.2468],[9.6604,50.232],[9.5808,50.2201],[9.5001,50.2419],[9.5018,50.212],[9.5196,50.1972],[9.5029,50.1933],[9.5039,50.1746],[9.5325,50.1674],[9.5109,50.1485],[9.5194,50.0925],[9.4192,50.0803],[9.3773,50.1322],[9.2379,50.1493],[9.1999,50.1375],[9.2063,50.1228],[9.1695,50.0888],[9.1732,50.1185],[9.1384,50.1252],[9.0338,50.1121],[9.0171,50.0934],[8.9954,50.0451],[9.0481,50.0424],[9.049,50.0056],[9.0667,49.9948],[9.0319,49.9905],[9.0393,49.9096],[9.0632,49.8695],[9.0459,49.8655],[9.0576,49.8584],[9.0463,49.8472],[9.0752,49.8476],[9.0697,49.8328],[9.1041,49.8443],[9.0872,49.822],[9.105,49.7928],[9.1453,49.7969],[9.1171,49.7597],[9.1635,49.7454],[9.1295,49.7157],[9.1355,49.701],[9.0953,49.6933],[9.1135,49.6456],[9.0735,49.6232],[9.0739,49.6049],[9.1089,49.583],[9.0923,49.5461],[9.1333,49.5204],[9.1103,49.513],[9.1042,49.5315],[9.0694,49.535],[9.0469,49.5055],[8.9542,49.5121],[8.9546,49.4901],[8.9374,49.4826],[8.9552,49.4609],[8.9265,49.446],[8.9032,49.453],[8.8978,49.431],[8.8578,49.4049],[8.8404,49.4155],[8.8176,49.4041],[8.8002,49.4182],[8.8399,49.441],[8.8277,49.4771],[8.8621,49.4888],[8.8623,49.4742],[8.8796,49.4745],[8.9025,49.4894],[8.9021,49.5113],[8.8331,49.4991],[8.8267,49.5318],[8.7982,49.5166],[8.7291,49.522],[8.6818,49.5791],[8.7042,49.6017],[8.6868,49.6086],[8.6978,49.6272],[8.6004,49.6132],[8.595,49.5984],[8.6251,49.5517],[8.5915,49.525],[8.5572,49.5238],[8.469,49.5906],[8.429,49.5856],[8.3934,49.6176],[8.3619,49.6902],[8.4472,49.7224],[8.4861,49.7678],[8.4284,49.766],[8.3861,49.82],[8.3967,49.8499],[8.3493,49.8817],[8.3655,49.9192],[8.3468,49.9593],[8.2754,50.0165],[8.188,50.0325],[7.9868,49.975],[7.8877,49.9707],[7.8687,50.0082],[7.7851,50.0531],[7.7846,50.0681],[7.8131,50.087],[7.8483,50.0834],[7.8473,50.1097],[7.87,50.1286],[7.9411,50.1025],[7.9341,50.1326],[7.9456,50.1402],[7.8976,50.1699],[7.9196,50.2037],[8.0063,50.2346],[8.0479,50.2125],[8.0764,50.2317],[8.0406,50.2537],[8.0517,50.2688],[8.1278,50.2589],[8.1389,50.2741],[8.1254,50.3262],[8.0901,50.3331],[8.0828,50.3742],[7.9842,50.4136],[8.0072,50.429],[7.9942,50.4399],[8.03,50.4482],[7.9966,50.4847],[8.011,50.5261],[8.0565,50.5572],[8.1151,50.5358],[8.162,50.5513],[8.1736,50.6003],[8.1408,50.6117],[8.1588,50.6191],[8.1562,50.6342],[8.1187,50.6757],[8.1392,50.6989],[8.1531,50.6948],[8.1799,50.7358],[8.1319,50.7918],[8.1789,50.8068],[8.2757,50.8812],[8.2999,50.8812],[8.3203,50.8588],[8.3805,50.859],[8.419,50.9],[8.4715,50.9115],[8.468,50.9633],[8.5547,51.0132],[8.5355,51.039],[8.5175,51.0387],[8.5404,51.0578],[8.5152,51.076],[8.5382,51.0951],[8.6699,51.0903],[8.7346,51.1065],[8.7098,51.1321],[8.7678,51.1706],[8.7668,51.1968],[8.7844,51.2046],[8.7407,51.2485],[8.7522,51.26],[8.7094,51.2741],[8.6017,51.2458],[8.569,51.2751],[8.5744,51.2864],[8.6325,51.3361],[8.6694,51.3368],[8.7038,51.3749],[8.9576,51.3873],[8.9686,51.4285],[8.9387,51.4279],[8.9378,51.4578],[8.9125,51.4797],[8.9905,51.5073],[9.0419,51.5193],[9.06,51.5011],[9.1062,51.4982],[9.096,51.4609],[9.1139,51.4426],[9.1773,51.4438],[9.1996,51.4666],[9.2287,51.4597],[9.239,51.4896],[9.3188,51.5135],[9.3286,51.5544],[9.379,51.5924],[9.3435,51.6138],[9.435,51.6303],[9.4861,51.6571],[9.5037,51.6537],[9.4988,51.6315]]]}
hessen_geometry = ee.Geometry.Polygon(hesse_JSON['coordinates'])

# Rainfall Stations
Alsfeld_Eifa = ee.Geometry.Point([9.3450,50.7446]) #Number 91 
Aßlar_Klein_Altensteadten = ee.Geometry.Point([8.4685,50.5800]) #Number 211
Biblis = ee.Geometry.Point([8.4179, 49.6896]) #Number 476
stations = ee.FeatureCollection([Alsfeld_Eifa,Aßlar_Klein_Altensteadten,Biblis])
stations_geometry = stations.geometry().getInfo()
stations_geometry = ee.Geometry.MultiPoint(stations_geometry['coordinates'])
stations_JSON = stations_geometry.toGeoJSON()

#Get geometry and convert it to GeoJSON for visualisation purpose 
ried_geometry = ried_225_222.geometry().getInfo() #dict object
ried_geometry = ee.Geometry.Polygon(ried_geometry['coordinates']) #cast to Geometry object
ried_JSON = ried_geometry.toGeoJSON() #get GeoJSON object

#get area of aoi
ried_area = ried_225_222.geometry().area().getInfo()
print("Hessisches Ried: Naturräumliche Einheiten 225 und 222, area: ca. ", ried_area/1000**2, "km²") #m² in km² 

#map properties
center, zoom = (49.7252978589571, 8.34580993652344), 9

#create basemap
map_aoi = Map(center = center, zoom = zoom, basemap=basemaps.Esri.NatGeoWorldMap)

#add Layer to map
ried_viz = GeoJSON(data = ried_JSON, style = {'color': 'blue', 'opacity':1, 'fillOpacity':0.25}) #Create GeoJSON ipyleaflet object
hesse_viz = GeoJSON(data = hesse_JSON, style = {'color': 'red', 'opacity':1, 'fillOpacity':0.25}) #Create GeoJSON ipyleaflet object
stations_viz = GeoJSON(data = stations_JSON, style = {'color': 'blue'}) #Create GeoJSON ipyleaflet object

map_aoi.add_layer(ried_viz) #add layer to map
map_aoi.add_layer(hesse_viz) #add layer to map
map_aoi.add_layer(stations_viz)

#display map
map_aoi


## Test the radolan projection by compare precipitation data from rainfall measure stations 

In [ ]:
#load radolan test files
I1 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090528').reduceRegion(ee.Reducer.mean(),Biblis).getInfo()
I2 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090529').reduceRegion(ee.Reducer.first(),Biblis).getInfo()
I3 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090530').reduceRegion(ee.Reducer.first(),Biblis).getInfo()
I4 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090531').reduceRegion(ee.Reducer.first(),Biblis).getInfo()
I5 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090601').reduceRegion(ee.Reducer.first(),Biblis).getInfo()
I6 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090602').reduceRegion(ee.Reducer.first(),Biblis).getInfo()
I7 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090603').reduceRegion(ee.Reducer.first(),Biblis).getInfo()
I8 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090604').reduceRegion(ee.Reducer.first(),Biblis).getInfo()
I9 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090605').reduceRegion(ee.Reducer.mean(),Biblis).getInfo()
I10 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090606').reduceRegion(ee.Reducer.mean(),Biblis).getInfo()
I11 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090607').reduceRegion(ee.Reducer.mean(),Biblis).getInfo()
I12 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090608').reduceRegion(ee.Reducer.mean(),Biblis).getInfo()
I13 = ee.Image('users/tillmueller1990/radolan/dwd_radolan_09-19/090609').reduceRegion(ee.Reducer.mean(),Biblis).getInfo()

#station data
biblis_prep = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.4,9.3,0.2,0.4]
biblis_radolan = [I1['b1'],I2['b1'],I3['b1'],I4['b1'],I5['b1'],I6['b1'],I7['b1'],I8['b1'],I9['b1'],I10['b1'],I11['b1'],I12['b1'],I13['b1']]
print(biblis_prep)
print(biblis_radolan)
#[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19.0, 0.4, 9.3, 0.2, 0.4]
#0.0, 1.2999999523162842, 0.0, 0.0, 0.10000000149011612, 0.0, 0.0, 0.0, 0.0, 5.699999809265137, 5.900000095367432, 2.700000047683716, 1.600000023841858]
#[0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.700000000000001, 10.9, 1.6, 3.8000000000000003]

# Remote sensed data

This work try to obtain the most data from osm remotely sensed products or self derived data.  
There are many properties affecting soil moisture:  
1. Soil type  (X)
2. Organic content  ( )
3. Precipitation  (X)
4. Vegetation  (X)
5. Altitude  (X)
6. Sun radiation (X)
7. Cloudcover  ( )
8. Soil temperature (X)
9. Air temperature (X)
10. Evaporation / Evapotranspiration (X)
11. Wind relation (X)
12. ? ( )

A brief overview of available datasets:  

### Earth Engine Datasets
#### NASA-USDA Global Soil Moisture Data https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_soil_moisture
revesit : 3days ; resoltion : 0.25 arc degrees (~25km)
Surface soil moisture  
Subsurface soil moisture  
Soil moisture profile  
Surface soil moisture anomaly  
Subsurface soil moisture anomaly  
#### CHIRPS Daily: Climate Hazards Group InfraRed Precipitation with Station Data https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY
revesit : daily ; resolution : 0.05 arc degrees (~5km)  
precipitation mm 
####  GLDAS-2.1: Global Land Data Assimilation System https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H
revesit : 3 H , resolotion : 0.25 arc degrees (~25km)  
Albedo  
Average surface skin temperature  
Plant canopy surface water  
Canopy water evaporation  
Direct evaporation from bare soil  
Evapotranspiration  
Downward long-wave radiation flux  
Net long-wave radiation flux  
Potential evaporation rate  
Pressure 
Specific humidity  
Heat flux  
Sensible heat net flux  
Latent heat net flux  
Storm surface runoff  
Baseflow-groundwater runoff  
Snow melt  
Total precipitation rat  
Rain precipitation rate  
Root zone soil moisture  
Snow depth water equivalent  
Downward short-wave radiation flux  
Snow depth  
Snow precipitation rate  
Soil moisture  
Soil temperature  
Net short wave radiation flux  
Air temperature  
Transpiration  
Wind speed  

####  Corine Land Cover 
Year: 2012, resolution: 100m

### Other Datasets

#### Bodenübersichtskarte von Hessen http://bodenviewer.hessen.de/mapapps/resources/apps/bodenviewer/index.html?lang=de
Year: 1997, scale: 1 : 500 000 bis zu 1:5000 




# Soil Type
Soil type is important because different kind of soils have different potentials to release water at same weather conditions.   
![Image of Yaktocat](http://www.soilmanagementindia.com/wp-content/uploads/2017/03/clip_image002-5.jpg)  

I found no source for downloading a soil map in any format there are only wms layer and bodenviewer of HLNUG so i take a screenshot from HLNUG Bodenviewe.  
The smallest scale fits on one screenshot. The other two scales (1:50.000 and 1:5.000) need several screenshots. Than you have to stich the images with e.g. autosticht together.  

In [ ]:
IFrame(src='http://bodenviewer.hessen.de/mapapps/resources/apps/bodenviewer/index.html?lang=de&center=550000%2C5595000%2C25832&lod=3', width=1400, height=600)

## All Band Classifier (KMeans)
Save this images as .tiff or .png to preserve the 3 bands. Because we need to make a KMeans classification and with more bands the classification is better. The number of possible soil classes are 32 + 1 for the white background. 

In [ ]:
filepath = r'C:\Users\USER\Desktop\master-thesis-master\soil_map\results\great_hesse_soil_map_ref.tif'

#Load file in gdal
file = gdal.Open(filepath, gdal.GA_ReadOnly)

#Initialize an array of zeros with size of input
img = np.zeros((file.RasterYSize, file.RasterXSize, file.RasterCount), gdal_array.GDALTypeCodeToNumericTypeCode(file.GetRasterBand(1).DataType))

#Loop over the number of bands in the image (img.shape[2]) insert values into the numpy array
for b in range(img.shape[2]):
    img[ : , : , b] = file.GetRasterBand(b + 1).ReadAsArray()

#Reshape array 
new_shape = (img.shape[0] * img.shape[1], img.shape[2])

#based on this shape, we can build the input value
X = img[: , : , :3].reshape(new_shape)
#X = np.where(X==150, 255, X) Bad idea because other r ,g or b values may have same as black and gray
#X = np.where(X==0, 255, X) 

#Classifier Kmeans
k_means = cluster.KMeans(n_clusters=33)
k_means.fit(X)

X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(img[:, :, 0].shape)

#Get statistic values
for i in np.unique(X_cluster):
    blobs = np.int_(morphology.binary_opening(X_cluster == i))
    color = np.around(k_means.cluster_centers_[i])
    count = len(np.unique(measure.label(blobs))) - 1
    print('Color: {}  >>  Objects: {}'.format(color, count))
    
#Plot the data
plt.figure(figsize=(20,10))
plt.imshow(X_cluster)

plt.show()


## Save Classification as GeoTiff 

In [ ]:
#Open input image again to get properties
file = gdal.Open(filepath)

#We need a single band
band = file.GetRasterBand(1)

#Convert it to an array
arr = band.ReadAsArray()

#Extract columns and rows to a list
[cols, rows] = arr.shape

#Set the output as Geotiff
format = "GTiff"
driver = gdal.GetDriverByName(format)

#Creates the output raster with the dimensions of the input raster
#outDataRaster = driver.Create("soil_map_hesse_classify_33.tiff", rows, cols, 1, gdal.GDT_Byte) -> activate this cell for saving image local

#Set the projection and extent of the data.
outDataRaster.SetGeoTransform(file.GetGeoTransform())##sets same geotransform as input
outDataRaster.SetProjection(file.GetProjection())##sets same projection as input

#Write classification result to a single band raster image 
outDataRaster.GetRasterBand(1).WriteArray(X_cluster)

#Calling FlushCache() to remove from memory and delete the data
outDataRaster.FlushCache() ## remove from memory
del outDataRaster ## delete the data (not the actual geotiff)

## Analyse Soil Types within Hessisches Ried

Create Features for different soil types to compare soil-moisture. 
Die Patches müssen in 1km² große Patches unterteilt werden damit man den Niederschlagsfehler so klein wie möglich hält. 

In [ ]:
#Load Soil Types from BÜK 1:500.000 and clip to AOI
BUK500_ried = ee.Image('users/tillmueller1990/soil_map_hesse_classify_33').clip(ried_225_222)

#Correct raster values to corresponding soil class values # Some Values have same classes: raste Value 5 == 9+23, 0 == 8,33 Lösung: Bestimmte Patches durch eine Andere Farbe ersetzten?!
raster_values = [0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31] 
viewer_values = [8,36,2,30,9,10,25,5,7,19,35,13,21,1,29,12,24,18,4,22,11,16,6,26,34,20,27,17,28,37,3]
BUK500_ried = BUK500_ried.remap(raster_values, viewer_values).select(['remapped'],['b1']) # remap and rename bandNAme to previous name

#Mask out Background // Not necessary if we use step above, then all values that are not listet get masked 
#mask = soil_map_great.neq(1) #Value 1 is invalid its the background (white) from screenshot or values with more than 1 soil class 
#soil_map_great = soil_map_great.updateMask(mask) #Apply Mask

#Get Unique values to check if every value is signed correctly 
BUK500_ried_uniqueValues = BUK500_ried.reduceRegion(reducer=ee.Reducer.countDistinct(), geometry=ried_geometry, scale= 265 ,maxPixels= 1e10)
print("BUK500_ried_uniqueValues", BUK500_ried_uniqueValues.getInfo())

#get area for each class
class_areas_ried = ee.Image.pixelArea().divide(1000*1000).addBands(BUK500_ried).reduceRegion(reducer= ee.Reducer.sum().group(groupField= 1,groupName= 'Soil class'),geometry=ried_geometry,scale= 265, maxPixels= 1e10).get('groups') # sample the geometry at 1m intervals;
#pp.pprint(class_areas.getInfo())

#clip to clc landcover classes to avoid measuering over citys, forest or other dense vegetation
soil_map_great_ried_clc = BUK500_ried.clip(clc_vector_ried)

#Convert an image to a feature collection by reducing homogenous regions ->>>>> Hier nochmal nachschauen man muss die ganzen winzigen Features rausfiltern ....
soil_map_great_ried_labeled = soil_map_great_ried_clc.connectedComponents(ee.Kernel.plus(255),250) #Finds connected components with the same value of the first band of the input and labels them with a globally unique value.
soil_map_great_ried_scale = soil_map_great_ried_labeled.projection().nominalScale().getInfo() #get scale parameter 
features_ried_clc = soil_map_great_ried_labeled.reduceToVectors(reducer = ee.Reducer.first(), geometry= ried_geometry, scale= 265, geometryType= 'Polygon', eightConnected= False,maxPixels= 1e10) #FeatureCollection of patches

def add_area_to_FC(feature):
    return feature.set('area',  feature.geometry().area(1).divide(1000*1000))

features_ried_clc = features_ried_clc.map(add_area_to_FC)
pp.pprint(features_ried_clc.size().getInfo())

#Grouped a count reducer: count number of land cover category pixels by patches
count_ried = soil_map_great_ried_labeled.select(['labels','b1']).reduceRegion(reducer= ee.Reducer.count().group(groupField= 0, groupName= 'label'), geometry= ried_geometry, scale= 265,  maxPixels= 1e8)

#Get test field for biggest area of soil class
features_ried_clc = features_ried_clc.filterMetadata('area','greater_than',0.1)
pp.pprint(features_ried_clc.size().getInfo())

#Make random points for Features in Read
random_ried = ee.FeatureCollection.randomPoints(features_ried_clc, 500)
pp.pprint(random_ried.size().getInfo())

#make sample polygon for ried with size of 1km²
image_feature_ried = features_ried_clc.reduceToImage(['label'], ee.Reducer.first())
#sample_regions_ried_clc_1km2 = image_feature_ried.reduceToVectors(ee.Reducer.first(), geometry, scale, geometryType, eightConnected, labelProperty, crs, crsTransform, bestEffort, maxPixels, tileScale, geometryInNativeProjection)

#radolan polygons clipped to crop land over ried
AOI_Features = ee.FeatureCollection('users/tillmueller1990/AOI_Features_clc_ried').filterBounds(ried_225_222)

#Create LookUp Table for raster values corresponding to the soil type 
soil_types_legend = {1:'Niedermoore, Hochmoore', 2:'Vega, Auengleye, örtl. Anmoorgleye', 3:'Tschernoseme', 4:'Parabraunerden', 5:'Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden', 7:'Pararendzinen, Braunerden mit Bändern, örtl. Bänder-Parabraunerden', 8:'Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden', 9:'Pararendzinen', 13: 'Parabraunerden, örtl. Pseudogley-Parabraunerden',25: 'Braunerden, Ranker-Braunerden, Regosol-Braunerden' , 34 : 'Braunerden, Braunerde-Pseudogleye, örtl. Podsol-Braunerden'} #Legende
raster_value_soil_value = {14:1, 3:2, 31:3, 19:4, 25:34, 7:25, 9:7 ,8:5, 9:7 , 0:8, 5:9 } # Value in raster value of legend

soil_classes = ['Niedermoore, Hochmoore','Vega, Auengleye, örtl. Anmoorgleye','Tschernoseme','Parabraunerden','Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden','Lockerbraunerden','Pararendzinen, Braunerden mit Bändern, örtl. Bänder-Parabraunerden','Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden','Pararendzinen','Parabraunerden, Tschernoseme','Tschernosem-Parabraunerden, Parabraunerden, örtl. Pseudogley-Parabraunerden','Parabraunerden, örtl. Pseudogley-Parabraunerden und Tschernosem-Parabraunerden','Parabraunerden, örtl. Pseudogley-Parabraunerden','Parabraunerden und Pseudogleye','Pseudogley-Parabraunerden, Pseudogleye, Parabraunerden','Braunerden, Parabraunerden','Braunerden, örtl. Pseudogleye','Pseudogley-Parabraunerden, Pseudogleye, Braunerden','Pseudogley-Braunerden, Pseudogley-Parabraunerden, Pseudogleye','Pseudogleye mit Übergängen zu Stagnogleyen','Rendzinen','Rendzinen, Braunerden, Pelosole','Braunerden, Ranker-Braunerden, Regosol-Braunerden','Braunerden','Fersiallite, örtl. Ferralite','Braunerden, Pelosole, Rendzinen','Braunerden, Pelosole','Braunerden, Pseudogleye, vereinzelt Podsol-Braunerden','Braunerden, örtl. Podsol-Braunerden und Pseudogley-Braunerden','Braunerden, Braunerde-Pseudogleye, örtl. Podsol-Braunerden','Braunerden, Ranker-Braunerden, örtl. Braunerde-Podsole','Braunerde-Podsole, Podsole','Podsol-Braunerden, Pseudogley-Parabraunerden, örtl. Podsole']


In [ ]:
#Export FeatureCollection to Earth Engine Assests
#ee.batch.Export.table.toAsset(features_ried_clc, 'Connected features from Ried analyse','users/tillmueller1990/ried_features_connected').start()
ee.batch.Export.table.toDrive(collection=features_ried_clc, fileFormat ="SHP").start()

## Analyse SoilMoisture within fields of different soil types in Hesse



In [ ]:
#Load Soil Types from BÜK 1:500.000 and clip to AOI
BUK500 = ee.Image('users/tillmueller1990/soil_map_hesse_classify_33')
BUK500_test = ee.Image('users/tillmueller1990/soil_map_hesse_classify_33')

#Correct raster values to corresponding soil class values # Some Values have same classes: raste Value 5 == 9+23, 0 == 8,33 Lösung: Bestimmte Patches durch eine Andere Farbe ersetzten?!
raster_values = [2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31] 
viewer_values = [36,2,30,10,25,5,7,19,35,13,21,1,29,12,24,18,4,22,11,16,6,26,34,20,27,17,28,37,3]
print(len(raster_values),len(viewer_values))
BUK500 = BUK500.remap(raster_values, viewer_values).select(['remapped'],['b1']) # remap and rename bandNAme to previous name

#Mask out Background // Not necessary if we use step above, then all values that are not listet get masked 
#mask = soil_map_great.neq(1) #Value 1 is invalid its the background (white) from screenshot or values with more than 1 soil class 
#soil_map_great = soil_map_great.updateMask(mask) #Apply Mask

#Get Unique values to check if every value is signed correctly 
BUK500_uniqueValues = BUK500.reduceRegion(reducer=ee.Reducer.countDistinct(), geometry=hessen_geometry, scale= 265 ,maxPixels= 1e10)
print("BUK500_uniqueValues", BUK500_uniqueValues.getInfo())

#get area for each class
class_areas = ee.Image.pixelArea().divide(1000*1000).addBands(BUK500).reduceRegion(reducer= ee.Reducer.sum().group(groupField= 1,groupName= 'Soil class'),geometry=hessen_geometry,scale= 265, maxPixels= 1e10).get('groups') # sample the geometry at 1m intervals;
#pp.pprint(class_areas.getInfo())

#clip to clc landcover classes to avoid measuering over citys, forest or other dense vegetation
soil_map_great_clc = BUK500.clip(clc_vector_hesse)

#Convert an image to a feature collection by reducing homogenous regions ->>>>> Hier nochmal nachschauen man muss die ganzen winzigen Features rausfiltern ....
soil_map_great_labeled = soil_map_great_clc.connectedComponents(ee.Kernel.plus(255),250) #Finds connected components with the same value of the first band of the input and labels them with a globally unique value.
soil_map_great_scale = soil_map_great_labeled.projection().nominalScale().getInfo() #get scale parameter 
features_clc = soil_map_great_labeled.reduceToVectors(reducer = ee.Reducer.first(), geometry= hessen_geometry, scale= 265, geometryType= 'Polygon', eightConnected= False,maxPixels= 1e10) #FeatureCollection of patches

def add_area_to_FC(feature):
    return feature.set('area',  feature.geometry().area(1).divide(1000*1000))
#Feature Collection with Area attributes
features_clc = features_clc.map(add_area_to_FC)

#Grouped a count reducer: count number of land cover category pixels by patches
count = soil_map_great_labeled.select(['labels','b1']).reduceRegion(reducer= ee.Reducer.count().group(groupField= 0, groupName= 'label'), geometry= hessen_geometry, scale= 265,  maxPixels= 1e8)

#Get test field for biggest area of soil class
features_clc = features_clc.filterMetadata('area','greater_than',0.1) #--> hier weiter machen zu viele patches 
pp.pprint(features_clc.size().getInfo())
#pp.pprint(features_clc.getInfo())

#Create LookUp Table for raster values corresponding to the soil type 
soil_types_legend = {1:'Niedermoore, Hochmoore', 2:'Vega, Auengleye, örtl. Anmoorgleye', 3:'Tschernoseme', 4:'Parabraunerden', 5:'Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden', 7:'Pararendzinen, Braunerden mit Bändern, örtl. Bänder-Parabraunerden', 8:'Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden', 9:'Pararendzinen', 13: 'Parabraunerden, örtl. Pseudogley-Parabraunerden',25: 'Braunerden, Ranker-Braunerden, Regosol-Braunerden' , 34 : 'Braunerden, Braunerde-Pseudogleye, örtl. Podsol-Braunerden'} #Legende
raster_value_soil_value = {14:1, 3:2, 31:3, 19:4, 25:34, 7:25, 9:7 ,8:5, 9:7 , 0:8, 5:9 } # Value in raster value of legend

soil_classes = ['Niedermoore, Hochmoore','Vega, Auengleye, örtl. Anmoorgleye','Tschernoseme','Parabraunerden','Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden','Lockerbraunerden','Pararendzinen, Braunerden mit Bändern, örtl. Bänder-Parabraunerden','Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden','Pararendzinen','Parabraunerden, Tschernoseme','Tschernosem-Parabraunerden, Parabraunerden, örtl. Pseudogley-Parabraunerden','Parabraunerden, örtl. Pseudogley-Parabraunerden und Tschernosem-Parabraunerden','Parabraunerden, örtl. Pseudogley-Parabraunerden','Parabraunerden und Pseudogleye','Pseudogley-Parabraunerden, Pseudogleye, Parabraunerden','Braunerden, Parabraunerden','Braunerden, örtl. Pseudogleye','Pseudogley-Parabraunerden, Pseudogleye, Braunerden','Pseudogley-Braunerden, Pseudogley-Parabraunerden, Pseudogleye','Pseudogleye mit Übergängen zu Stagnogleyen','Rendzinen','Rendzinen, Braunerden, Pelosole','Braunerden, Ranker-Braunerden, Regosol-Braunerden','Braunerden','Fersiallite, örtl. Ferralite','Braunerden, Pelosole, Rendzinen','Braunerden, Pelosole','Braunerden, Pseudogleye, vereinzelt Podsol-Braunerden','Braunerden, örtl. Podsol-Braunerden und Pseudogley-Braunerden','Braunerden, Braunerde-Pseudogleye, örtl. Podsol-Braunerden','Braunerden, Ranker-Braunerden, örtl. Braunerde-Podsole','Braunerde-Podsole, Podsole','Podsol-Braunerden, Pseudogley-Parabraunerden, örtl. Podsole']


In [ ]:
#create basemap
map_s = Map(center = center, zoom = zoom, basemap=basemaps.Esri.NatGeoWorldMap)

#soil color map
soil_color_dict_ried = {1:'87FF4B', 2:'B5E6F5', 3:'D8A56D', 4:'C1994B', 5:'EDB74B', 7:'FFD1EE', 8:'FFE0A5', 9:'E99DFF',13:'FFBA43',25:'FFC3D1',34:'EECEA1'}
soil_color_list_ried = list(soil_color_dict_ried.values())
soil_color_dict = {1:'87FF4B', 2:'B5E6F5', 3:'D8A56D', 4:'C1994B', 5:'EDB74B', 6:'D579D1', 7:'FFD1EE', 10:'C1A771', 11:'D7AD98', 12:'DEA071', 13:'FFBA43', 16:'DBDB93', 17:'D9BE83', 18:'AC8A7B', 19:'CDA08C', 20:'E2CEAC', 21:'D7D1B1', 22:'DBDBDB', 24:'EED1FF', 25:'FFC3D1', 26:'FFD29D', 27:'FFA5A5', 28:'DB93B7', 29:'F7A1CB', 30:'E7BFA2', 34:'EECEA1', 35:'F7D86F', 36:'EDDD4B', 37:'FFE483'}
soil_color_list = list(soil_color_dict.values())

#add Layer to map
soil_map_great_ried_viz = GetTileLayerUrl(BUK500_ried.visualize(min=0, max=10, palette=soil_color_list_ried)) #Create GeoJSON ipyleaflet object
soil_map_great_ried_layer = ipyleaflet.TileLayer(url=soil_map_great_ried_viz,name='soil_map_great_ried')

soil_map_great_ried_patches_viz = GetTileLayerUrl(features_ried_clc.draw(color='red').visualize())
soil_map_great_ried_patches_layer = ipyleaflet.TileLayer(url=soil_map_great_ried_patches_viz, name='soil_class features_ried')

soil_map_great_viz = GetTileLayerUrl(BUK500.visualize(min=0, max=29, palette=soil_color_list)) #Create GeoJSON ipyleaflet object
soil_map_great_layer = ipyleaflet.TileLayer(url=soil_map_great_viz,name='soil_map_great')

soil_map_great_patches_viz = GetTileLayerUrl(features_clc.draw(color='red').visualize())
soil_map_great_patches_layer = ipyleaflet.TileLayer(url=soil_map_great_patches_viz, name='soil_class features')

random_ried_viz = GetTileLayerUrl(random_ried.draw(color='blue').visualize())
random_ried_layer = ipyleaflet.TileLayer(url=random_ried_viz, name='random points ried')

image_feature_ried_viz = GetTileLayerUrl(image_feature_ried.visualize())
image_feature_ried_layer = ipyleaflet.TileLayer(url=image_feature_ried_viz, name='image feature ried')

AOI_Features_viz = GetTileLayerUrl(AOI_Features.draw(color='blue').visualize())
AOI_Features_layer = ipyleaflet.TileLayer(url=AOI_Features_viz, name='AOI Features Ried')

map_s.add_layer(soil_map_great_ried_layer) #add layer to map
map_s.add_layer(soil_map_great_ried_patches_layer) #add layer to map
map_s.add_layer(soil_map_great_layer) #add layer to map
map_s.add_layer(soil_map_great_patches_layer) #add layer to map
map_s.add_layer(random_ried_layer)
map_s.add_layer(image_feature_ried_layer)
map_s.add_layer(AOI_Features_layer)

#display map
map_s.add_control(LayersControl())
map_s



In [ ]:
#Make Histogram
def get_histogram_from_image(image,geometry):
    dict_histogram = image.reduceRegion(reducer=ee.Reducer.autoHistogram(), geometry=geometry, scale= 265,maxPixels= 1e10).getInfo()['b1']
    keys = [x[0] for x in dict_histogram if x[1] > 0.0]  #get keys from [key,value] list if value is > 0
    values = [x[1] for x in dict_histogram if x[0] in keys] #get values for keys if the key is in value
    return dict(zip(keys,values))

histogram_soil_map_great_ried = get_histogram_from_image(BUK500_ried,ried_geometry)
print("soil_map_great_ried_histogram", histogram_soil_map_great_ried)

plt.bar(histogram_soil_map_great_ried.keys(), histogram_soil_map_great_ried.values(), color='g')
plt.title("Histogram soil class for ried")
plt.show()

In [ ]:
histogram_soil_map_great = get_histogram_from_image(BUK500,hessen_geometry)
print("soil_map_great_histogram", histogram_soil_map_great)

plt.bar(histogram_soil_map_great.keys(), histogram_soil_map_great.values(), color='g')
plt.title("Histogram soil class")
plt.show()

## Soil moisture from Sentinel 1 
Here i try to derive soil moisture content from SAR data hostet on Google Earth Engine. The algorithm is a simple Change Detection Method more to this later....

In [ ]:
#Sentinel 1 backscatter raw data convertet to reflectivity range between 0 and 1 indicator for high reflectivity = high water volume content and vis verse 
#Load Sentinel 1 and filter data
def load_dataset(ImageCollection_ID,begin,end,aoi):
    ic = ee.ImageCollection(ImageCollection_ID).filterDate(begin,end).filterBounds(aoi)
    return ic

def filter_sentinel1(ImageCollection,polarisation,instrumentMode,resolution):
    ic = ImageCollection.filter(ee.Filter.listContains('transmitterReceiverPolarisation',polarisation)).filter(ee.Filter.eq('instrumentMode',instrumentMode)).filterMetadata('resolution_meters','equals', resolution)
    return ic

def seperate_look_angels(ImageCollection):
    Ascending = ImageCollection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    Descending = ImageCollection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    return Ascending,Descending

def seperate_tiles(ImageCollection,tiles):
    tile_list = [x]

def get_geometry(ImageCollection):
    geometry = ImageCollection.geometry().getInfo()
    #geometry = [x for x in geometry['coordinates']]
    return geometry

def show_tiles(ImageCollection):
    geometry = get_geometry(ImageCollection)
    geometry_list = list(geometry['coordinates'])
    flattened_list = [y for x in geometry_list for y in x] #De flatter list 
    unique_list = []
    unique_list = [x for x in flattened_list if x not in unique_list]
    print("different tiles: ",len(unique_list))
    m = Map(center=(49.6252978589571, 8.34580993652344), zoom=7)
    geo_json = GeoJSON(data=geometry, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
    m.add_layer(geo_json)
    dc = ipyleaflet.DrawControl()
    m.add_control(dc)
    m.add_control(LayersControl())
    return m
    
def get_properties(ImageCollection):
    features = ImageCollection.getInfo()['features']
    dict_list = []
    for f in features:
        prop = f['properties']
        dict_list.append(prop)
    df = pd.DataFrame.from_records(dict_list).drop(['system:footprint','transmitterReceiverPolarisation'],axis=1)
    #Pandas Series of unique distinc values in df
    unique = df.nunique()
    im_id_list = [item.get('id') for item in ImageCollection.getInfo().get('features')]
    date_list = [datetime.datetime.strptime(x[35:43],'%Y%m%d') for x in im_id_list]
    #property_names = list(df.columns.values) 
    return unique, im_id_list, date_list

def make_mosaic(date,ImageCollection):
    date = ee.Date(date['value'])
    filterCollection = ImageCollection.filterDate(date, date.advance(1,'day'))
    #Make the mosaic
    image = ee.Image(filterCollection.mosaic()).copyProperties(filterCollection.first(),["system:time_start"])
    #Add the mosaic to a list only if the collection has images
    #return ee.List(ee.Algorithms.If(filterCollection.size(), newList.add(image), newList))
    return image

def clip_aoi(ImageCollection):
    im = ImageCollection.clip(ried_225_222)
    return im

def add_area(image):
    area = image.multiply(ee.Image.pixelArea()).divide(-(1000*1000))
    stat = area.reduceRegion(ee.Reducer.sum(),ried_225_222,10) 
    im = image.set('area',stat.get('VV'))
    return im

def reproject(image):
    VV = image.select('VV')
    return image.reporject({crs: VV.projection().crs(), scale : 100})

def toNatural(image):
    return ee.Image(10.0).pow(image.select(0).divide(10.0)).copyProperties(image)

def toDB(image):
    return ee.Image(image).log10().multiply(10.0).copyProperties(image)

#Sigma Lee speckle filteringThe RL speckle filter from https://code.earthengine.google.com/2ef38463ebaf5ae133a478f173fd0ab5 by Guido Lemoine
def RefinedLee(img):
    #img must be in natural units, i.e. not in dB!
    #Set up 3x3 kernels
    weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
    kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)
    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)
    #Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])
    sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)
    #Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)
    #Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())
    #And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())
    #Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)
    #duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)
    #Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
    #The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))
    #Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)
    #"collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum())
    #Generate stats
    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))
    #Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])
    #Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))
    #Set weights
    diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0],[1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])
    rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)
    #Create stacks for mean and variance using the original kernels Mask with relevant direction.
    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))
    dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(),diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(),diag_kernel).updateMask(directions.eq(2)))
    #and add the bands for rotated kernels
    for i in range(4):
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(),rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(),rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(),diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(),diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
    #"collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())
    #A finally generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))
    b = varX.divide(dir_var)
    result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
    return result.arrayFlatten([['VV']]).copyProperties(img)

def calc_asc_soilMoisture(image):
    im = image.expression('(omega - omegaD) / (omegaW - omegaD)', {'omega' : image.select('VV'), 'omegaD' : minMax_asc.select('VV_max'), 'omegaW' : minMax_asc.select('VV_min')})
    return im.addBands(im).select(['VV','VV_1'],['Backscatter_Coefficient','soil_moisture_content'])

def calc_des_soilMoisture(image):
    im = image.expression('(omega - omegaD) / (omegaW - omegaD)', {'omega' : image.select('VV'), 'omegaD' : minMax_des.select('VV_max'), 'omegaW' : minMax_des.select('VV_min')})
    return im.addBands(im).select(['VV','VV_1'],['Backscatter_Coefficient','soil_moisture_content'])

def filter_IC(ImageCollection,filter):
    old_size = ImageCollection.size().getInfo()
    new_coll = ImageCollection.filter(filter)
    new_size = new_coll.size().getInfo()
    return new_coll

def reducer(ImageCollection,reducer):
    im = ImageCollection.reduce(reducer)
    return im

def plot_image(ImageCollection):
    m = Map(center=(49.6252978589571, 8.34580993652344), zoom=7)
    ic = GetTileLayerUrl(ImageCollection.first().visualize())
    m.add_layer(ic)
    dc = ipyleaflet.DrawControl()
    m.add_control(dc)
    m.add_control(LayersControl())
    return m

def windy_days_filter(image):
    d = image.date().format('Y-M-d')
    wx = ee.ImageCollection('NOAA/CFSV2/FOR6H')
    vWind = wx.select(['v-component_of_wind_height_above_ground'])
    a = vWind.max()
    uWind = wx.select(['u-component_of_wind_height_above_ground'])
    b = uWind.max()
    a = a.pow(2)
    b = b.pow(2)
    ab = a.add(b)
    ws = ab.sqrt()
    ws = ws.multiply(3.6)
    return image.updateMask(ws.lt(12))

#Time of interest
begin = ee.Date.fromYMD(2013,1,1)
end = ee.Date.fromYMD(2019,6,1)
date_range = end.difference(begin, 'day')

#Source dataset
sentinel1 = load_dataset('COPERNICUS/S1_GRD',begin,end,ried_225_222)
print("sentinel1",type(sentinel1),"Collection Size: ", sentinel1.size().getInfo())

#Filter dataset for High resolution and Vertical transmitt vertical receive
sentinel1_VV = filter_sentinel1(sentinel1,'VV','IW',10)
print("sentinel1_VV",type(sentinel1_VV),"Collection Size: ", sentinel1_VV.size().getInfo())

print(sentinel1_VV.first().propertyNames().getInfo())
#The RL speckle filter from https://code.earthengine.google.com/2ef38463ebaf5ae133a478f173fd0ab5 by Guido Lemoine
sentinel1_VV_natural = sentinel1_VV.map(toNatural)
#print(sentinel1_VV_natural.first().propertyNames().getInfo())
sentinel1_VV_slFilter = sentinel1_VV_natural.map(RefinedLee)
#print(sentinel1_VV_slFilter.first().propertyNames().getInfo())                      
sentinel1_VV = sentinel1_VV_slFilter.map(toDB)
#print(sentinel1_VV.first().bandNames().getInfo())
#print(sentinel1_VV.first().propertyNames().getInfo())

#Filter for different look angles
VV_Ascending,VV_Descending = seperate_look_angels(sentinel1_VV)
print("VV_Ascending",type(VV_Ascending),"VV_Descending",type(VV_Descending),"Collection Size: ", VV_Ascending.size().getInfo(), VV_Descending.size().getInfo())

#Clip images to AOI and calculate area property
VV_aoi_asc = VV_Ascending.map(clip_aoi).map(add_area)
print("VV_aoi_asc",type(VV_aoi_asc),"Collection Size: ", VV_aoi_asc.size().getInfo())
VV_aoi_des = VV_Descending.map(clip_aoi).map(add_area)
print("VV_aoi_des",type(VV_aoi_des),"Collection Size: ", VV_aoi_des.size().getInfo())

#Create Min and Max bands for change detection method
minMax_asc = reducer(VV_aoi_asc,ee.Reducer.minMax())
print("minMax_asc",type(minMax_asc),minMax_asc.getInfo())
minMax_des = reducer(VV_aoi_des,ee.Reducer.minMax())
print("minMax_des",type(minMax_des),minMax_des.getInfo())

#Compute soil moisture with simple change detection Methode
VV_asc_sm = VV_aoi_asc.map(calc_asc_soilMoisture)
print("VV_asc_sm",type(VV_asc_sm),"Collection Size: ", VV_asc_sm.size().getInfo())
VV_des_sm = VV_aoi_des.map(calc_des_soilMoisture)
print("VV_des_sm",type(VV_des_sm),"Collection Size: ", VV_des_sm.size().getInfo())

### Get compareable test fields of soil classes
Because it is not usefull to measure the sm content in areas of city, urban and forest we have to find bare or crop land or other spare vegetation land.  
To find these areas there are two methods:  
1. Is to detect these landclasses by a indice like ndvi, ndbi...., 
2. use existing data products like the corine land cover classification 

## Corine Land Cover (CLC) 

In [ ]:
# Load last corine land cover image 
clc = ee.Image.load('COPERNICUS/CORINE/V18_5_1/100m/2012', -1)

# Mask areas where soil moisture measurements valid (farmland cat.:11-16)
clc_mask = clc.gte(11).And(clc.lte(16)) #binary map for updateMask
clc = clc.updateMask(clc_mask) #set mask for not Farmland

# Clip to extend of Hessisches Ried
clc_image_ried = clc.clip(ried_225_222) #image

# Feature Collection of clc farmland cat.: 11-16 with 350m resolution
clc_vector_ried = clc.reduceToVectors(geometry=ried_225_222, crs=clc.projection(), scale=100, geometryType='polygon', eightConnected = True) #FeatureCollection
clc_vector_hesse = clc.reduceToVectors(geometry=hessen_geometry, crs=clc.projection(), scale=100, geometryType='polygon', eightConnected = True) #FeatureCollection

# Data Extraction

## Precipitation

In [ ]:
# ==========================================================================
# Function to Convert Feature Classes to Pandas Dataframe
# Adapted from: https://events.hpc.grnet.gr/event/47/material/1/12.py https://gis.stackexchange.com/questions/257727/iterate-over-imagecollection-returning-pandas-dataframe-using-earth-engine-pyt
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        ##attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = pd.DataFrame(dictarr)
    # Convert GeoJSON features to shape
    #print(df['geometry'])
    #df['geometry'] = map(lambda s: shape(s), df.geometry)
    #print(df['geometry'])
    return df

# ==========================================================================
# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, features_aoi):
    IC = data_set.filterMetadata('system:index', 'equals', img_id)
    image = IC.reduce(ee.Reducer.mean())
    fc_image_redR = image.reduceRegions(collection=features_aoi,
                                  reducer = ee.Reducer.mean(),
                                  scale = 1000,
                                  crs = proj)

    # Convert to Pandas Dataframe
    df_image_red = fc2df(fc_image_redR)

    # Add Date as Variable
    df_image_red['date'] = datetime.datetime.strptime(img_id[:],'%y%m%d')
    return df_image_red

# ==========================================================================
#### Features
features_f = AOI_Features

#### Projection
proj = ee.Projection('EPSG:4326')

#### Load Raster
data_set = radolan

#### Make list of image IDs and Dates
data_id = [item['properties']['system:index'] for item in data_set.getInfo().get('features')]
data_date = [datetime.datetime.strptime(x[:],'%y%m%d') for x in data_id] #http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html

#### Create Initial Pandas Dataframe
df_all = extract_point_values(data_id[0], features_f)
df_all = df_all.drop([0,1])

#### Iterate over all impages
for i in data_id:
    df_all = df_all.append(extract_point_values(i, features_f))

#save dataframe
df_all.to_csv('radolan_1km2_AOI_Features.csv')

#### Display Results
df_all  

## Soil Moisture ASC/DESC

### SM ASC

In [ ]:
# ==========================================================================
# Function to Convert Feature Classes to Pandas Dataframe
# Adapted from: https://events.hpc.grnet.gr/event/47/material/1/12.py https://gis.stackexchange.com/questions/257727/iterate-over-imagecollection-returning-pandas-dataframe-using-earth-engine-pyt
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = gpd.GeoDataFrame(dictarr)
    # Convert GeoJSON features to shape
    #print(df['geometry'])
    df['geometry'] = map(lambda s: shape(s), df.geometry)
    #print(df['geometry'])
    return df

# ==========================================================================
# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, features_aoi):
    IC = data_set.filterMetadata('system:index', 'equals', img_id)
    image = IC.reduce(ee.Reducer.mean())
    fc_image_redR = image.reduceRegions(collection=features_aoi,
                                  reducer = ee.Reducer.mean(),
                                  scale = 1000,
                                  crs = proj)

    # Convert to Pandas Dataframe
    df_image_red = fc2df(fc_image_redR)

    # Add Date as Variable
    df_image_red['date'] = datetime.datetime.strptime(img_id[17:32],'%Y%m%dT%H%M%S')
    return df_image_red

# ==========================================================================
#### Features
features_f = AOI_Features

#### Projection
proj = ee.Projection('EPSG:4326')

#### Load Raster
data_set = VV_asc_sm.select('soil_moisture_content')

#### Make list of image IDs and Dates
data_id = [item['properties']['system:index'] for item in data_set.getInfo().get('features')]
data_date = [datetime.datetime.strptime(x[17:32],'%Y%m%dT%H%M%S') for x in data_id] #http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html

#### Create Initial Pandas Dataframe
df_all = extract_point_values(data_id[0], features_f)
df_all = df_all.drop([0,1])

#### Iterate over all impages
for i in data_id:
    df_all = df_all.append(extract_point_values(i, features_f),sort=True)

#save dataframe
df_all.to_csv('sm_asc_1km2_AOI_Features.csv')

#### Display Results
df_all  

In [ ]:
df_all.iloc[231]

### SM DESC

In [ ]:
# ==========================================================================
# Function to Convert Feature Classes to Pandas Dataframe
# Adapted from: https://events.hpc.grnet.gr/event/47/material/1/12.py https://gis.stackexchange.com/questions/257727/iterate-over-imagecollection-returning-pandas-dataframe-using-earth-engine-pyt
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = gpd.GeoDataFrame(dictarr)
    # Convert GeoJSON features to shape
    #print(df['geometry'])
    df['geometry'] = map(lambda s: shape(s), df.geometry)
    #print(df['geometry'])
    return df

# ==========================================================================
# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, features_aoi):
    IC = data_set.filterMetadata('system:index', 'equals', img_id)
    image = IC.reduce(ee.Reducer.mean())
    fc_image_redR = image.reduceRegions(collection=features_aoi,
                                  reducer = ee.Reducer.mean(),
                                  scale = 1000,
                                  crs = proj)

    # Convert to Pandas Dataframe
    df_image_red = fc2df(fc_image_redR)

    # Add Date as Variable
    df_image_red['date'] = datetime.datetime.strptime(img_id[17:32],'%Y%m%dT%H%M%S')
    return df_image_red

# ==========================================================================
#### Features
features_f = AOI_Features

#### Projection
proj = ee.Projection('EPSG:4326')

#### Load Raster
data_set = VV_des_sm.select('soil_moisture_content')

#### Make list of image IDs and Dates
data_id = [item['properties']['system:index'] for item in data_set.getInfo().get('features')]
data_date = [datetime.datetime.strptime(x[17:32],'%Y%m%dT%H%M%S') for x in data_id] #http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html

#### Create Initial Pandas Dataframe
df_all = extract_point_values(data_id[0], features_f)
df_all = df_all.drop([0,1])

#### Iterate over all impages
for i in data_id:
    df_all = df_all.append(extract_point_values(i, features_f),sort=True)

#save dataframe
df_all.to_csv('sm_des_1km2_AOI_Features.csv')

#### Display Results
df_all  

In [ ]:
# ==========================================================================
# Function to Convert Feature Classes to Pandas Dataframe
# Adapted from: https://events.hpc.grnet.gr/event/47/material/1/12.py https://gis.stackexchange.com/questions/257727/iterate-over-imagecollection-returning-pandas-dataframe-using-earth-engine-pyt
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = gpd.GeoDataFrame(dictarr)
    # Convert GeoJSON features to shape
    #print(df['geometry'])
    df['geometry'] = map(lambda s: shape(s), df.geometry)
    #print(df['geometry'])
    return df

# ==========================================================================
# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, pts):
    IC = soil_moisture_1.filterMetadata('system:index', 'equals', img_id)
    image = IC.reduce(ee.Reducer.mean())
    fc_image_redR = image.reduceRegions(collection=pts,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs = proj)

    # Convert to Pandas Dataframe
    df_image_red = fc2df(fc_image_redR)

    # Add Date as Variable
    df_image_red['date'] = datetime.datetime.strptime(img_id[:],'%y%m%d')
    return df_image_red

# ==========================================================================
#### Features
features_f = features_ried_clc

#### Projection
proj = ee.Projection('EPSG:4326')

#### Load Raster
soil_moisture_1 = radolan

#### Make list of image IDs and Dates
sm_1_id = [item['properties']['system:index'] for item in soil_moisture_1.getInfo().get('features')]
sm_1_date = [datetime.datetime.strptime(x[:],'%y%m%d') for x in sm_1_id] #http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html

#### Create Initial Pandas Dataframe
df_all = extract_point_values(sm_1_id[0], features_f)
df_all = df_all.drop([0,1])

#### Iterate over all impages
for i in sm_1_id:
    df_all = df_all.append(extract_point_values(i, features_f))

#### Display Results
df_all  

In [ ]:
#save dataframe
df_all.to_csv('radolan_1k_ried_clc.csv')


## Precipitation - RADOLAN


In [ ]:
radolan = ee.ImageCollection('users/tillmueller1990/radolan/dwd_radolan_09-19').filterDate("2014-1-1","2019-05-31")
print(radolan.size().getInfo())
print(radolan.first().bandNames().getInfo())
print(radolan.first().date().getInfo())

# GLDAS 

In [ ]:
GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
start,end = datetime.datetime(2014,1,1),datetime.datetime(2019,6,1)
GLDAS = GLDAS.filterDate(start,end).filterMetadata('start_hour', 'equals', 18) #Daten für 18 Uhr if not than you have more than 5000 Elements
print(GLDAS.first().get('system:index').getInfo())


# Inspect data

In [ ]:
#Load data
sm_asc = pd.read_csv(r"C:\Users\USER\Desktop\Masterarbeit\DATA\master_data\GEE_data\sm_s1_asc_250_ried_clc.csv")
sm_des = pd.read_csv(r"C:\Users\USER\Desktop\Masterarbeit\DATA\master_data\GEE_data\sm_s1_des_250_ried_clc.csv")
radolan = pd.read_csv(r"C:\Users\USER\Desktop\Masterarbeit\DATA\master_data\GEE_data\radolan_1k_ried_clc.csv")
gldas = pd.read_csv(r"C:\Users\USER\Desktop\Masterarbeit\DATA\master_data\GEE_data\GLDAS_25k_ried_clc.csv") 

#Delete invalid Geometry column
soil_moisture_asc = sm_asc.drop(['area','label'], axis=1)
soil_moisture_des = sm_des.drop(['geometry','area','label'], axis=1)
precipitation = radolan.drop(['geometry','area','label'],axis=1)
gldas_values = gldas.drop(['geometry','area','label'],axis=1)

#rename column 
soil_moisture_asc.rename(columns={'Unnamed: 0':'Feature','first':'Soil class','mean':'sm_asc'}, inplace=True)
soil_moisture_des.rename(columns={'Unnamed: 0':'Feature','first':'Soil class','mean':'sm_des'}, inplace=True)
precipitation.rename(columns={'Unnamed: 0' : 'Feature', 'first' : 'Soil class','mean':'precipitation'}, inplace=True)
#https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H ---> possible bands
gldas_values.rename(columns={'Unnamed: 0' : 'Feature', 'first' : 'Soil class','Albedo_inst_mean':'Albedo',
                           'AvgSurfT_inst_mean':'Average surface skin temperature','Evap_tavg_mean':'Evapotranspiration','ESoil_tavg_mean':'Direct evaporation from bare soil',
                           'ECanop_tavg_mean':'Canopy water evaporation','Evap_tavg_mean':'Evapotranspiration',"PotEvap_tavg_mean":"Potential evaporation rate",
                           }, inplace=True)

#merge tabels
model = precipitation
#print(model.count,model.shape)
model.insert(3,'sm_asc',soil_moisture_asc['sm_asc'],True)
#print(model.count,model.shape)
model.insert(4,'sm_des',soil_moisture_des['sm_des'],True)
#print(model.count,model.shape)
model.insert(5,'Potential evaporation rate',gldas_values['Potential evaporation rate'],True)
#print(model.count,model.shape)

# descriptions
print(model.describe())



#index dates
#soil_moisture_asc.index = soil_moisture_asc['date']

#gldas_values

In [ ]:
# class distribution
#print(model.groupby('Feature').size())

# box and whisker plots
model.plot(kind='box', subplots=True, layout=(3,3), sharex=False, sharey=False)
plt.show()

# histograms
model.hist()
plt.show()

In [ ]:
#Load Datasets
soil_moisture_asc = pd.read_csv(r'C:\Users\USER\Desktop\master-thesis-master\sm_s1_des_250_ried_clc.csv') #Load from CSV file
soil_moisture_asc = soil_moisture_asc.drop(['geometry'], axis=1) #Delete invalid Geometry column
soil_moisture_asc.rename(columns={'Unnamed: 0':'Feature'}, inplace=True) #rename column 
soil_moisture_asc.rename(columns={'first':'Soil Class'}, inplace=True) #rename column 
soil_moisture_asc.index = soil_moisture_asc['date']
del soil_moisture_asc['date']
soil_moisture_asc_small = soil_moisture_asc[(soil_moisture_asc['Feature'] >= 1) & (soil_moisture_asc['Feature'] <=30 )]
soil_moisture_asc_grouped = soil_moisture_asc.groupby('Soil Class')
soil_moisture_asc_grouped

In [ ]:
radolan = ee.ImageCollection('users/tillmueller1990/radolan/radolan_dwd_germany')
chirps  = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').map(clip_aoi)
soil_moisture_asc_ried_soil = chirps.toArray().reduceRegion(reducer=ee.Reducer.mean(), geometry=ried_geometry, scale=1000)
soil_moisture_asc_ried_soil.getInfo()

In [ ]:
def rename_attribute(obj, old_name, new_name):
    obj.__dict__[new_name] = obj.__dict__.pop(old_name)
    



#Correct raster values to corresponding soil class values 
soil_map_great = soil_map_great.where(soil_map_great.eq(14),1) ##
soil_map_great = soil_map_great.where(soil_map_great.eq(12),13) ##
soil_map_great = soil_map_great.where(soil_map_great.eq(2),36) ##
soil_map_great = soil_map_great.where(soil_map_great.eq(3),2) ##
soil_map_great = soil_map_great.where(soil_map_great.eq(31),3) ##
soil_map_great = soil_map_great.where(soil_map_great.eq(19),4) ##
soil_map_great = soil_map_great.where(soil_map_great.eq(25),34) ## 
soil_map_great = soil_map_great.where(soil_map_great.eq(7),25) ## FFC3D1,Braunerden, Ranker-Braunerden, Regosol-Braunerden
soil_map_great = soil_map_great.where(soil_map_great.eq(9),7) ## 
soil_map_great = soil_map_great.where(soil_map_great.eq(5),9) ## 5 = 9+23 
soil_map_great = soil_map_great.where(soil_map_great.eq(8),5) ## 
soil_map_great = soil_map_great.where(soil_map_great.eq(0),8) ## 0=33(FFE0A5)(Braunerden, örtl. Podsol-Braunerden und Pseudogley-Braunerden) und 8 (Braunerden mit Bändern, Bänder-Parabraunerden, örtl. Podsol-Braunerden)
soil_map_great = soil_map_great.where(soil_map_great.eq(8),5) ## 


#Raster,Viewer,Color,class,Ausgangsgestein 33
2,36,EDDD4B,Podsol-Braunerden, Pseudogley-Parabraunerden, örtl. Podsole
23,6,D579D1,Lockerbraunerden,Trachytische Aschen
28,17,D9BE83,Pseudogley-Parabraunerden, Pseudogleye, Parabraunerden;Lösslehm mit Gesteinsbeimengungen
20,22,DBDBDB,Pseudogleye mit Übergängen zu Stagnogleyen,Lösslehm über dichtem Untergrund
13,21,D7D1B1,Pseudogley-Braunerden, Pseudogley-Parabraunerden, Pseudogleye,vorwiegend Lösslehm mit Gesteinsbeimengungen
4,30,E7BFA2,Braunerden, Pseudogleye, vereinzelt Podsol-Braunerden,Tonschiefer, Grauwackenschiefer, Phyllit
6,10,C1A771,Parabraunerden, Tschernoseme;Löss
21,11,D7AD98,Tschernosem-Parabraunerden, Parabraunerden, örtl. Pseudogley-Parabraunerden;Löss
16,12,DEA071,Parabraunerden, örtl. Pseudogley-Parabraunerden und Tschernosem-Parabraunerden;Löss
22,16,DBDB93,Parabraunerden und Pseudogleye;Lösslehm, örtl. mit Gesteinsbeimengungen
18,18,AC8A7B,Braunerden, Parabraunerden;Basalt, Lösslehm, Löss
10,19,CDA08C,Braunerden, örtl. Pseudogleye;Basalt, Lösslehm
26,20,E2CEAC,Pseudogley-Parabraunerden, Pseudogleye, Braunerden;Lösslehm, Basalt
5,23,E89CFF,Rendzinen;Kalkstein, Mergel, Dolomit # == 5 but class 9 == val. 5
17,24,EED1FF,Rendzinen, Braunerden, Pelosole;Kalkstein, Mergel, Dolomit, Ton- und Schluffsteine und Arkosen
24,26,FFD29D,Braunerden;Schalstein, Diabas
27,27,FFA5A5,Fersiallite, örtl. Ferralite;Basalt, Basalttuff
29,28,DB93B7,Braunerden, Pelosole, Rendzinen;Ton- und Schluffsteine, Arkosen, Kalkstein, Mergel, Dolomit
15,29,F7A1CB,Braunerden, Pelosole;Ton- und Schluffsteine und Arkosen, örtl. carbonathaltig
0,33,FFE0A5,Braunerden, örtl. Podsol-Braunerden und Pseudogley-Braunerden;Schluff- und Tonsteine, Sandsteine
11,35,F7D86F,Braunerden, Ranker-Braunerden, örtl. Braunerde-Podsole;Grauwacken, Sandsteine, Konglomerate, Quarzite, Kieselschiefer
30,37,FFE483,Braunerde-Podsole, Podsole;Quarzite, Sandsteine


